# Criação de Dashboard sobre Vendas de Supermercados

- Fonte: *Supermarket sales* - Kaggle (https://www.kaggle.com/aungpyaeap/supermarket-sales)
- Objetivo: Criação de *dashboard* que gerem *insights* quanto as vendas dos supermercados apresentados de modo a auxiliar, por exemplo, o desenvolvimento de promoções e outras campanhas de marketing.

> Este notebook depende do *dataset* mantido pelo usuário *Aung Pyae* na plataforma kaggle. Baixe o dataset no site indicado acima (requer *login*) e o desempacote na mesma pasta que este notebook.

## Importando as bibliotecas e acessando os dados

In [1]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

data = pd.read_csv('supermarket_sales - Sheet1.csv')
data.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


Segundo informações fornecidas pelo mantenedor do dataset, as colunas referem-se a:

- Invoice ID: número de identificação da fatura;
- Branch: filiais do supermercado, identificadas por A, B e C. 
- City: cidade me que estão localizados os supermercados.
- Customer type: tipo de cliente. Member se utilizou cartão de membro ou Normal caso contrário.
- Gender: sexo do cliente.
- Product line: categoria dos produtos comprados. Divididos em:
    - Eletronic accessories;
    - Fashion accessories;
    - Food and beverages;
    - Health and beauty;
    - Home and lifestyle;
    - Sports and travel.
- Unit price: preço unitário do produto.
- Quantity: quantidade de produtos comprados.
- Tax: valor da taxa de 5% sobre a compra.
- Total: preço total, incluindo as taxas.
- Date: data da compra. (dados disponíveis entre Janeiro de 2019 à Março de 2019)
- Time: horário da compra.
- Payment: forma como foi feito o pagamento. 3 opções disponíveis:
    - Cash;
    - Credit Card;
    - Ewallet.
- COGS: custo dos bens vendidos.
- Gross margin percent: percentual da margem bruta.
- Gross income: lucro bruto.
- Rating: avaliação do cliente de sua experiência de compra. Em escala de 1 a 10.

## Planejamento das Visualizações

Com base na observação dos dados e no objetivo do dashboard resultante, foram definidas, em ordem de relevância, as seguintes visualizações:
- Gráfico de linhas, mostrando a quantidade de vendas de cada categoria de produtos ao longo dos 3 meses de dados coletados. (opções para substituir vendas por: valor total dos produtos e lucro bruto obtido; também para mostrar os dados de cada filial/cidade separadamente).
- Gráfico de barras empilhadas com as porcentagens de vendas por: sexo, tipo de cliente (membro ou não) e tipo de pagamento efetuado. (opção para cada categoria de produtos separadamente, assim como por filial/cidade).
- Gráfico de frequencia dos periodos (horas) em que mais ocorreram vendas. (opções por dia de semana, por filial/cidade e por categoria de produtos).
- Gráfico de frequencia das avaliações dos clientes. (opção por filial/cidade e por categoria de produtos).

## Pré-processamento dos dados

In [2]:
data.shape

(1000, 17)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  gross margin percentage  

In [4]:
data.describe()

,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000
mean,55.672130,5.510000,15.379369,322.966749,307.58738,4.761905,15.379369,6.97270
std,26.494628,2.923431,11.708825,245.885335,234.17651,0.000000,11.708825,1.71858
min,10.080000,1.000000,0.508500,10.678500,10.17000,4.761905,0.508500,4.00000
25%,32.875000,3.000000,5.924875,124.422375,118.49750,4.761905,5.924875,5.50000
50%,55.230000,5.000000,12.088000,253.848000,241.76000,4.761905,12.088000,7.00000
75%,77.935000,8.000000,22.445250,471.350250,448.90500,4.761905,22.445250,8.50000
max,99.960000,10.000000,49.650000,1042.650000,993.00000,4.761905,49.650000,10.00000


Observações iniciais:
- O *dataset* não apresenta dados faltantes.
- Nem todas as colunas são relevantes à aplicação a ser criada.
- A coluna *Time* possui valores entre 10am e 9pm (segundo mantenedor do dataset). É mais interessante transformar-la em categorias, como "10:00 às 11:00", "11:00 às 12:00", "12:00 às 13:00", e assim em diante, de forma a determinar os horários em que os clientes mais compraram nos supermercados. 

In [5]:
# Select relevant data
data = data[
    ['City',
     'Customer type',
     'Gender', 
     'Product line',
     'Quantity',
     'Total',
     'Date',
     'Time',
     'Payment',
     'gross income',
     'Rating']
]

# Convert Date to datetime object
data['Date'] = pd.to_datetime(data['Date'])

# Binning time column
hours = pd.to_datetime(data['Time'], format='%H:%M').dt.hour
data['Time'] = pd.cut(hours, 
                    bins=[9,10,11,12,13,14,15,16,17,18,19,20,21], 
                    include_lowest=True, 
                    right=False,
                    labels=[
                        '9 to 10',
                        '10 to 11',
                        '11 to 12',
                        '12 to 13',
                        '13 to 14',
                        '14 to 15',
                        '15 to 16',
                        '16 to 17',
                        '17 to 18',
                        '18 to 19',
                        '19 to 20',
                        '20 to 21',
                    ])
data.head()

,City,Customer type,Gender,Product line,Quantity,Total,Date,Time,Payment,gross income,Rating
0,Yangon,Member,Female,Health and beauty,7,548.9715,2019-01-05,13 to 14,Ewallet,26.1415,9.1
1,Naypyitaw,Normal,Female,Electronic accessories,5,80.2200,2019-03-08,10 to 11,Cash,3.8200,9.6
2,Yangon,Normal,Male,Home and lifestyle,7,340.5255,2019-03-03,13 to 14,Credit card,16.2155,7.4
3,Yangon,Member,Male,Health and beauty,8,489.0480,2019-01-27,20 to 21,Ewallet,23.2880,8.4
4,Yangon,Normal,Male,Sports and travel,7,634.3785,2019-02-08,10 to 11,Ewallet,30.2085,5.3


## App Layout

In [6]:
# App layout
colors = {
    'background': '#453032',
    'text': '#ffffff', #'#bfbf88',
    'title': '#e62e24',
    'bg1': '#60354f',
    'bg2': '#c2b7a9',
    'bg3': '#807670',
    'bg4': '#93867a'
}

app.layout = html.Div(
    style={
        'backgroundColor': colors['background'],
        'color': colors['text'],
        'border-radius': '25px',
        'display': 'grid',
        'grid-template-columns': '1fr 1fr 1fr 1fr 1fr 1fr',
        'grid-template-rows': '.1fr .5fr .5fr',
        'grid-template-areas': '"title title title title title title" "timeline timeline timeline timeline percent percent" "times times times ratings ratings ratings"',
        'gap': '5px 5px',
        'align-items': 'start'
    }, children=[

    html.Div(style={'grid-area': 'title'}, children=[
        html.Link(rel='preconnect', href='https://fonts.gstatic.com'),
        html.Link(rel='stylesheet', href='https://fonts.googleapis.com/css2?family=Lobster+Two:ital@1&display=swap'),
        
        html.H1('Supermarket Sales', style={'text-align': 'center', 'color': colors['title'], 'font-family': 'Lobster Two'}),
    ]),
        
    # Sales over time
    html.Div(
        style={
            'grid-area': 'timeline',
            'margin-left': '5px',
            'padding': '5px',
            'backgroundColor': colors['bg1']
        }, children=[
            html.Div(style={'display': 'flex', 'align-items': 'center'}, children=[
                html.Label('Variable: '),
                dcc.Dropdown(id='slct_var_over_time',
                             options=[
                                 {'label': 'Sales', 'value': 'sales'},
                                 {'label': 'Total Spent', 'value': 'total'}],
                             multi=False,
                             value='sales',
                             style={'width': "50%", 'color': '#000000'}
                             ),
    
                html.Label('City: '),
                dcc.Dropdown(id='slct_over_time_city',
                             options=[
                                 {'label': 'All', 'value': 'All'},
                                 {'label': 'Yangon', 'value': 'Yangon'},
                                 {'label': 'Mandalay', 'value': 'Mandalay'},
                                 {'label': 'Naypyitaw', 'value': 'Naypyitaw'}],
                             multi=False,
                             value='All',
                             style={'width': '50%', 'color': '#000000'}
                             ),
                
                html.Label('Product Line: '),
                dcc.Dropdown(id='slct_over_time_product',
                             options=[
                                 {'label': 'All', 'value': 'All'},
                                 {'label': 'Electronic Accessories', 'value': 'Electronic accessories'},
                                 {'label': 'Food and Beverages', 'value': 'Food and beverages'},
                                 {'label': 'Health and Beauty', 'value': 'Health and beauty'},
                                 {'label': 'Home and Lifestyle', 'value': 'Home and lifestyle'},
                                 {'label': 'Sports and Travel', 'value': 'Sports and travel'}],
                             multi=False,
                             value='All',
                             style={'width': "50%", 'color': '#000000'}
                             ),

            ]),
            
            html.Br(),
            dcc.Graph(id='over_time_graph', figure={})
    ]),
    
    # Gender, Customer Type and Payment Option
    html.Div(
        style={
            'grid-area': 'percent',
            'margin-right': '5px',
            'padding': '5px',
            'backgroundColor': colors['bg2']
        }, children=[
            html.Div(style={'display': 'flex', 'align-items': 'center'}, children=[
                html.Label('Product Line: '),
                dcc.Dropdown(id='slct_percent_product',
                             options=[
                                 {'label': 'All', 'value': 'All'},
                                 {'label': 'Eletronic Accessories', 'value': 'eletronic'},
                                 {'label': 'Food and Beverages', 'value': 'food'},
                                 {'label': 'Health and Beauty', 'value': 'health'},
                                 {'label': 'Home and Lifestyle', 'value': 'home'},
                                 {'label': 'Sports and Travel', 'value': 'sports'}],
                             multi=False,
                             value='All',
                             style={'width': "70%", 'color': '#000000'}
                             ),

                html.Label('City: '),
                dcc.Dropdown(id='slct_percent_city',
                             options=[
                                 {'label': 'All', 'value': 'All'},
                                 {'label': 'Yangon', 'value': 'Yangon'},
                                 {'label': 'Mandalay', 'value': 'Mandalay'},
                                 {'label': 'Naypyitaw', 'value': 'Naypyitaw'}],
                             multi=False,
                             value='All',
                             style={'width': "70%", 'color': '#000000'}
                             ),
            ]),

        html.Br(),
        dcc.Graph(id='percent_graph', figure={})
    ]),
    
    # Sales times
    html.Div(
        style={
            'grid-area': 'times',
            'margin-left': '5px',
            'margin-bottom': '25px',
            'padding': '5px',
            'backgroundColor': colors['bg3']
        }, children=[
            html.Div(style={'display': 'flex', 'align-items': 'center'}, children=[
                html.Label('Days of the Week: '),
                dcc.Dropdown(id='slct_times_days',
                             options=[
                                 {'label': 'Sunday', 'value': 'sunday'},
                                 {'label': 'Monday', 'value': 'monday'},
                                 {'label': 'Tuesday', 'value': 'tuesday'},
                                 {'label': 'Wednesday', 'value': 'wednesday'},
                                 {'label': 'Thursday', 'value': 'thursday'},
                                 {'label': 'Friday', 'value': 'friday'},
                                 {'label': 'Saturday', 'value': 'saturday'}],
                             multi=True,
                             value=['sunday', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday'],
                             style={'width': "100%", 'color': '#000000'}
                             ),
            ]),

            html.Div(style={'display': 'flex', 'align-items': 'center'}, children=[
                html.Label('Product Line: '),
                dcc.Dropdown(id='slct_times_product',
                             options=[
                                 {'label': 'All', 'value': 'All'},
                                 {'label': 'Eletronic Accessories', 'value': 'eletronic'},
                                 {'label': 'Food and Beverages', 'value': 'food'},
                                 {'label': 'Health and Beauty', 'value': 'health'},
                                 {'label': 'Home and Lifestyle', 'value': 'home'},
                                 {'label': 'Sports and Travel', 'value': 'sports'}],
                             multi=False,
                             value='All',
                             style={'width': "50%", 'color': '#000000'}
                             ),

                html.Label('City: '),
                dcc.Dropdown(id='slct_times_city',
                             options=[
                                 {'label': 'All', 'value': 'All'},
                                 {'label': 'Yangon', 'value': 'Yangon'},
                                 {'label': 'Mandalay', 'value': 'Mandalay'},
                                 {'label': 'Naypyitaw', 'value': 'Naypyitaw'}],
                             multi=False,
                             value='All',
                             style={'width': "50%", 'color': '#000000'}
                             ),
            ]),

        html.Br(),
        dcc.Graph(id='times_graph', figure={})
    ]),

    # Customer's Ratings
    html.Div(
        style={
            'grid-area': 'ratings',
            'margin-right': '5px',
            'margin-bottom': '25px',
            'padding': '5px',
            'backgroundColor': colors['bg4']
        }, children=[
            html.Div(style={'display': 'flex', 'align-items': 'center'}, children=[
                html.Label('Product Line: '),
                dcc.Dropdown(id='slct_ratings_product',
                             options=[
                                 {'label': 'All', 'value': 'All'},
                                 {'label': 'Eletronic Accessories', 'value': 'eletronic'},
                                 {'label': 'Food and Beverages', 'value': 'food'},
                                 {'label': 'Health and Beauty', 'value': 'health'},
                                 {'label': 'Home and Lifestyle', 'value': 'home'},
                                 {'label': 'Sports and Travel', 'value': 'sports'}],
                             multi=False,
                             value='All',
                             style={'width': "50%", 'color': '#000000'}
                             ),

                html.Label('City: '),
                dcc.Dropdown(id='slct_ratings_city',
                             options=[
                                 {'label': 'All', 'value': 'All'},
                                 {'label': 'Yangon', 'value': 'Yangon'},
                                 {'label': 'Mandalay', 'value': 'Mandalay'},
                                 {'label': 'Naypyitaw', 'value': 'Naypyitaw'}],
                             multi=False,
                             value='All',
                             style={'width': "50%", 'color': '#000000'}
                             ),
            ]),

        html.Br(),
        dcc.Graph(id='ratings_graph', figure={})
    ])
])

In [7]:
data[data['Product line'] == 'Sports and travel']

,City,Customer type,Gender,Product line,Quantity,Total,Date,Time,Payment,gross income,Rating
4,Yangon,Normal,Male,Sports and travel,7,634.3785,2019-02-08,10 to 11,Ewallet,30.2085,5.3
15,Mandalay,Member,Female,Sports and travel,6,590.4360,2019-01-15,16 to 17,Cash,28.1160,4.5
17,Yangon,Normal,Male,Sports and travel,6,457.4430,2019-01-01,10 to 11,Credit card,21.7830,6.9
24,Yangon,Member,Male,Sports and travel,3,279.1845,2019-03-02,17 to 18,Ewallet,13.2945,6.0
31,Mandalay,Member,Male,Sports and travel,9,737.7615,2019-01-28,12 to 13,Cash,35.1315,4.5
...,...,...,...,...,...,...,...,...,...,...,...
926,Mandalay,Member,Male,Sports and travel,1,92.7255,2019-02-15,17 to 18,Credit card,4.4155,5.2
929,Mandalay,Normal,Male,Sports and travel,2,53.1510,2019-03-02,19 to 20,Ewallet,2.5310,7.2
937,Yangon,Normal,Female,Sports and travel,5,469.7700,2019-03-30,10 to 11,Cash,22.3700,7.4
982,Yangon,Member,Female,Sports and travel,9,921.1860,2019-03-14,14 to 15,Ewallet,43.8660,7.4


In [8]:
@app.callback(
    [Output(component_id='over_time_graph', component_property='figure')],
    [Input(component_id='slct_var_over_time', component_property='value'),
    Input(component_id='slct_over_time_city', component_property='value'),
    Input(component_id='slct_over_time_product', component_property='value')]
)
def update_ot_graph(var, city, product):
    
    df = data.copy()
    
    if city != 'All':
        df = df[df['City'] == city]

    if product != 'All':
        df = df[df['Product line'] == product]
        
    if var == 'sales':
        df = df[['Date', 'Product line', 'Quantity']].groupby(['Date', 'Product line']).sum().reset_index().sort_values(by=['Date'])
        df = df.rename(columns={'Quantity': 'Sales', 'Product line': 'Product Line'})
        
        fig = px.line(
            df,
            x='Date',
            y='Sales',
            color='Product Line'
        )
        
    elif var == 'total':
        df = df[['Date', 'Product line', 'Total']].groupby(['Date', 'Product line']).sum().reset_index().sort_values(by=['Date'])
        df = df.rename(columns={'Total': 'Total Spent', 'Product line': 'Product Line'})

        fig = px.line(
            df,
            x='Date',
            y='Total Spent',
            color='Product Line'
        )

    return [fig]

In [ ]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Feb/2021 13:09:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:09:35] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:09:35] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:09:35] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:09:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:09:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:10:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:10:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:10:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:11:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:11:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2021 13:21:04] "POST /_dash-update-component H

# 